In [3]:
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import dictionary_corpus
from dictionary_corpus import Corpus
import numpy as np
from sklearn.decomposition import PCA, SparsePCA, KernelPCA, IncrementalPCA
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
import pickle as pk
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)

matplotlib.use('webagg')
# load the model and the corpus
model = torch.load('hidden650_batch128_dropout0.2_lr20.0.pt',map_location=torch.device('cpu'))
corpus = Corpus('')
# print("Vocab size %d", ntokens)
import matplotlib.rcsetup as rcsetup
print(rcsetup.all_backends)

['GTK3Agg', 'GTK3Cairo', 'GTK4Agg', 'GTK4Cairo', 'MacOSX', 'nbAgg', 'QtAgg', 'QtCairo', 'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 'WXAgg', 'WXCairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']


In [4]:
for word in corpus.dictionary.word2idx.keys():
    if corpus.dictionary.word2idx[word]==3:
        print(word)

the


In [204]:
def get_init_emb(l, model=model, corpus=corpus):
	# def initial emb feature extraction function
    df = pd.DataFrame(columns = ['labels','tensors'])
    for word in tqdm(l):
        with torch.no_grad():
            wordid = corpus.dictionary.word2idx[word]
            torch.save([1,2,3],'ip.npy')
            # initailize the hidden state of the model
            hidden = model.init_hidden(1)
            _, hid, emb = model(torch.as_tensor(wordid).reshape(1,1), hidden)
                # default export
    # export with `dynamic_axes`
            
            # the init embedding and the free_hid is the embedding we wanted.
#             emb = emb.view(650).numpy()
            emb = hid[1][1].view(650).numpy()
#             emb = hid[0][1].view(650).numpy()

            df0 = pd.DataFrame([[word,emb]], columns = ['labels','tensors'])
            df=df.append(df0,ignore_index=True)
    return df

In [207]:
# all_word_init=get_init_emb([word for word in corpus.dictionary.word2idx.keys()])

In [208]:
# with open('all_word_c1', 'wb') as f:
#      pk.dump(all_word_init, f)

In [8]:
# with open('all_word_emb', 'rb') as f:
#     x0 = pk.load(f)
with open('all_word_c1', 'rb') as f:
    x1 = pk.load(f)
# with open('all_word_c','rb') as f:
#     x2 = pk.load(f)

In [9]:
# y=x0.set_index('labels')
y2= x1.set_index('labels')
# y2 = x2.set_index('labels')
a=y2.loc['man']
b=y2.loc['woman']
c=y2.loc['man']
d=y2.loc['woman']
a1=y2.loc['France']
a2=y2.loc['Paris']
a3=y2.loc['Tokyo']
a4=y2.loc['Japan']

In [280]:
x = euclidean(y2.loc['woman']['tensors'],y2.loc['girl']['tensors'])

In [281]:
x

3.56

In [13]:
def l2_ranking(df):
    new = df.copy()
    new['l2'] = new['tensors'].apply(lambda x: np.linalg.norm(x)).round(2)
    return new

In [14]:
a=l2_ranking(y2)

In [15]:
# a.sort_values(by='l2').head(100)

In [16]:
with open('Swi','rb') as f:
    a = pk.load(f)
a['num'] = a.index
a['num']=a['num'].apply(lambda x: str(x))
a['labels'] = a['num']+a['prev']+a['target']
a = a.drop(['file','prev','num','target'],axis=1).set_index('labels')
y2 = pd.concat([y2,a])

In [17]:
#np.array(a.loc[i]['tensors'].to_list())
# y2.loc['k-q+w'] = v+d
# v = c-d
# y2.loc['m-w+q']=v+b
# y2.loc['f-p']=a1-a2
# y2.loc['j-t']=a4-a3

In [18]:
len(y2)

50036

In [19]:
def euclidean(x,y):
    dist = np.linalg.norm(x-y)
    return dist.round(2)
def cos(x,y):
    dist = np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    return dist.round(2)
def find_top_k_sim(dfk,k,tar):
    l = []
    v = dfk.loc[tar]['tensors']
    dfk['dist']= dfk['tensors'].apply(lambda x: euclidean(x,v))
    l = sorted(list(dfk['dist']))[::k]
    return dfk.sort_values(by='dist',ascending=True)[:k]

In [20]:
list1 = ['Russia','Italy','Austria','Japan','Britain','France','Germany','Poland','Spain','Portugal','couch','Sweden','India','England']
list2 = ['cat','cats','dog','dogs','chicken','chickens','lamb','lambs','cow','cows','duck','ducks','fish','fishes','salmon']
list3 = ['tomato','potato','rice','wheat','meat','corn','cucumber','lettuce','apple','cabbage','garlic','cotton','barley','grape']
list4 = ['0','2','3','4','5','6','7','8','9']
listman = ['any','ever']
lista = [w for w in a.index]
list5 = ['Switzerland','sofa','couch','high','low','up','chair','bed','<eos>']
# list6 = ['Germany','each','every','England','k-q+w','m-w+q','j-t','f-p']
list6 = ['5rt']
list7 = ['evaluated','examine','evaluating','evaluate','analyze','assess','examines']
list8 = ['see','hear','watch','go','do','form','lead','record','chicken','bank']
list9 = ['abilities','bank','can','kitchen','computer','powers','ducks','duck','fish','lamb','potato','power','pupil','rock','salmon','tomato','transistor','watch',]
listA = list1 + list2 +list3+list4+list7

In [21]:
# lista

In [26]:
# top 30 similar word with f+w
d = {}
for each in tqdm(listman):
    a = find_top_k_sim(y2,40,each)
    a['s'] = a['dist'].apply(lambda x: str(x)[:4])
    a['s'] = a.index+' '+a['s']
    d[each] =  list(a['s'])
dfmat = pd.DataFrame.from_dict(d)
dfmat

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  2.31it/s]


,any,ever
0,any 0.0,ever 0.0
1,Any 4.94,never 4.76
2,no 5.57,occasionally 4.84
3,some 5.59,briefly 4.94
4,another 5.88,also 4.94
5,certain 6.11,always 4.98
6,each 6.26,subsequently 5.09
7,sole 6.36,likewise 5.11
8,various 6.42,actually 5.13
9,many 6.48,reportedly 5.13


In [286]:
# find top similairty for a list of words
d = {}
for each in tqdm(list9):
    a = find_top_k_sim(y2,30,each)
    a['s'] = a['dist'].apply(lambda x: str(x)[:4])
    a['s'] = a.index+' '+a['s']
    d[each] =  list(a['s'])
dfmat = pd.DataFrame.from_dict(d)
dfmat

 44%|███████████████████▌                        | 8/18 [00:06<00:08,  1.16it/s]


KeyboardInterrupt: 

In [87]:
def word_heat(l,d0):
#   takes in a list of word and a df with the column dist
    d3 = d0.loc[l]
    print(len(d3))
    for word in d3.index:
        d3[word] = d3['tensors'].apply(lambda x: cos(x,d3.loc[word]['tensors']))
    return d3.drop(['tensors'],axis=1)

In [88]:
sim = word_heat(list2,y2)
sns.set_theme()
mask = np.zeros_like(sim)
mask[np.triu_indices_from(mask)] = True
f = plt.figure(figsize = (8.5,8.5)).add_subplot(1,1,1)
ax = sns.heatmap(sim,vmin=-0.2,vmax=1,mask=mask)
plt.title('list of words freeh')
plt.show()

15
Press Ctrl+C to stop WebAgg server


RuntimeError: This event loop is already running